In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, split
from pyspark.sql.types import StringType, IntegerType, FloatType
import pandas as pd
import html

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("Process Data").getOrCreate()

In [2]:
# Định nghĩa hàm để xử lý các ký tự Unicode
def replace_unicode(text):
    chars = []
    codes = []
    with open("D:\\.Repo\\Incomplete Project\\Thesis Project --- Spark\\unicode.txt", mode="r", encoding="utf8") as file:
        unicode_table = file.read().split("\n")
        for i in unicode_table:
            i = i.split()
            chars.append(i[0])
            codes.append(i[1])
    for i in range(len(chars)):
        text = text.replace(codes[i], chars[i])
    return text[:-2]

# Định nghĩa hàm để chuyển đổi HTML entities thành ký tự Unicode
def unescape_html(text):
    return html.unescape(text)

# Định nghĩa hàm để chuyển đổi định dạng ngày sinh
def process_dob(dob):
    return dob[:-2]

# Định nghĩa hàm để xử lý điểm thi và chuyển đổi về lower case
def process_score(score):
    score = score[:-2]
    score = replace_unicode(score)
    for i in range(len(score)):
        if score[i:i+2] == "&#":
            score = score[:i] + html.unescape(score[i:i+5]) + score[i+6:]
    score = score.replace(":", "")
#     return "".join(score)
    score = score.replace("KHXH ", "KHXH   ")
    score = score.replace("KHTN ", "KHTN   ")
    score_list = score.split("   ")
    data = []
    for subject in ["Toán", "Ngữ văn", "KHXH", "KHTN", "Lịch sử", "Địa lí", "GDCD", "Sinh học", "Vật lí", "Hóa học", "Tiếng Anh"]:
        if subject in score_list:
            data.append(str(float(score_list[score_list.index(subject) + 1])))  # Chuyen 7.00 -> 7.0
        else:
            data.append("")
    return " ".join(data)

# Định nghĩa hàm để chuyển đổi các trường dữ liệu về định dạng số
def to_float(value):
    if value.strip() == "":
        return None
    return float(value)

# Normal
def normal(txt):
    return txt

In [3]:
# Đọc dữ liệu từ file text và tạo DataFrame từ dữ liệu đó
data_rdd = spark.sparkContext.textFile("D:\\.Repo\\Incomplete Project\\Thesis Project --- Spark\\raw_data.txt")
data_df = data_rdd.filter(lambda line: len(line.strip()) > 0) \
    .map(lambda line: line.strip()) \
    .filter(lambda line: line.startswith("b'<!DOCTYPE html>")) \
    .map(lambda line: line.split("\\n")) \
    .filter(lambda arr: len(arr) == 90) \
    .map(lambda arr: (arr[61].strip(), arr[64].strip(), arr[67].strip())) \
    .toDF(["name", "dob", "score"])

In [4]:
# Áp dụng các hàm xử lý dữ liệu vào DataFrame
replace_unicode_udf = udf(replace_unicode, StringType())
unescape_html_udf = udf(unescape_html, StringType())
process_dob_udf = udf(process_dob, StringType())
process_score_udf = udf(process_score, StringType())
to_float_udf = udf(to_float, FloatType())
normal_udf = udf(normal, StringType())

In [5]:
processed_df = data_df.withColumn("name", replace_unicode_udf("name")) \
    .withColumn("name", unescape_html_udf("name")) \
    .withColumn("score", process_score_udf("score")) \
    .withColumn("dob", process_dob_udf("dob")) \
    .withColumn("toan", split(normal_udf("score"), " ")[0]) \
    .withColumn("ngu_van", split(normal_udf("score"), " ")[1]) \
    .withColumn("tieng_anh", split(normal_udf("score"), " ")[10]) \
    .withColumn("lich_su", split(normal_udf("score"), " ")[4]) \
    .withColumn("dia_ly", split(normal_udf("score"), " ")[5]) \
    .withColumn("gdcd", split(normal_udf("score"), " ")[6]) \
    .withColumn("khxh", split(normal_udf("score"), " ")[2]) \
    .withColumn("sinh_hoc", split(normal_udf("score"), " ")[7]) \
    .withColumn("vat_li", split(normal_udf("score"), " ")[8]) \
    .withColumn("hoc_hoc", split(normal_udf("score"), " ")[9]) \
    .withColumn("khtn", split(normal_udf("score"), " ")[3])

processed_df = processed_df.drop('score')

In [6]:
processed_df.show(2)

+-------------------+----------+----+-------+---------+-------+------+----+----+--------+------+-------+----+
|               name|       dob|toan|ngu_van|tieng_anh|lich_su|dia_ly|gdcd|khxh|sinh_hoc|vat_li|hoc_hoc|khtn|
+-------------------+----------+----+-------+---------+-------+------+----+----+--------+------+-------+----+
|PHẠM HOÀNG HƯƠNG ÁI|04/11/2002| 6.6|   6.25|      5.2|   5.75|   7.0|7.25|6.67|        |      |       |    |
| ĐẶNG HUỲNH VĨNH AN|13/12/2002| 8.2|   7.75|      7.0|    7.0|  7.25| 8.5|7.58|        |      |       |    |
+-------------------+----------+----+-------+---------+-------+------+----+----+--------+------+-------+----+
only showing top 2 rows



In [7]:
pd_df = processed_df.toPandas()
so_bao_danh = arr = ['0'+str(2000000+i) for i in range(1, len(pd_df)+1)]
pd_df.insert(loc=0, column='sbd', value=so_bao_danh)

In [8]:
pd_df.head()

,sbd,name,dob,toan,ngu_van,tieng_anh,lich_su,dia_ly,gdcd,khxh,sinh_hoc,vat_li,hoc_hoc,khtn
0,02000001,PHẠM HOÀNG HƯƠNG ÁI,04/11/2002,6.6,6.25,5.2,5.75,7.0,7.25,6.67,,,,
1,02000002,ĐẶNG HUỲNH VĨNH AN,13/12/2002,8.2,7.75,7.0,7.0,7.25,8.5,7.58,,,,
2,02000003,LÂM NGUYỄN MỘNG THÙY AN,06/04/2001,6.8,6.75,6.0,4.75,7.75,8.25,6.92,,,,
3,02000004,LÊ TIÊU HOÀNG AN,18/11/2002,7.8,6.25,5.6,,,,,7.0,5.5,6.25,6.25
4,02000005,LƯ THUẬN AN,14/01/2002,6.4,6.5,8.2,,,,,5.5,6.75,6.25,6.17


In [9]:
pd_df.tail()

,sbd,name,dob,toan,ngu_van,tieng_anh,lich_su,dia_ly,gdcd,khxh,sinh_hoc,vat_li,hoc_hoc,khtn
74448,02074449,NGUYỄN ĐÌNH HẢI YẾN,10/11/2002,5.4,5.92,,,,,,5.5,3.5,5.0,4.67
74449,02074450,NGUYỄN THỊ HẢI YẾN,05/05/2002,8.6,7.0,6.2,,,,,6.75,8.25,8.75,7.92
74450,02074451,NGUYỄN TRẦN TIỂU YẾN,07/10/2002,9.2,7.5,6.2,,,,,8.25,8.75,8.5,8.5
74451,02074452,PHẠM THỊ KIM YẾN,20/11/2002,9.0,7.0,7.0,,,,,6.75,8.0,9.25,8.0
74452,02074453,VĂN HỒNG YẾN,27/10/2002,5.6,6.5,,6.5,7.25,,,,,,


In [11]:
pd_df.to_csv("diem_thi.csv", index = False)